In [1]:
from PIL import Image, ImageOps
import numpy as np
import os
import glob
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm_notebook as tqdm
import shutil as sh
import random

In [2]:
home_dir = '/home/alex/ScreenReplay/synt_generator/generated'

In [18]:
a = SynthDocumentGenerator(home_dir, 10,10)
a.start_generation()

generate 2d...



generate 3d...


In [17]:
class SynthDocumentGenerator:
    
    def __init__(self, source_dir, count_2d, count_3d):
        
        self.source_dir = source_dir # папка с прямыми паспортами
        self.count_2d = int(count_2d) # количество прямых
        self.count_3d = int(count_3d) # количесвто с изгибом
        self.save_dir = os.path.join(source_dir,'save')       
        self.images = glob.glob(self.source_dir+'/*.jpg')           
           
    
    def _random_background(self): 
        
        " создание фонов произвольной ргб заливки"

        a = 1080 # размер фона
        b = 1920
        zero_area = (np.zeros((b,a,3), dtype = "uint8"))
        Image.fromarray(zero_area)
        start_point = (0, 0) 
        end_point = (a, b) 
        color = (random.randint(0,255), random.randint(0,255), random.randint(0,255))
        thickness = -1
        res = cv2.rectangle(zero_area, start_point, end_point, color, thickness) 
        res = Image.fromarray(res)
        return res
    
    def _get_concat_v(self,im1, im2):

        'складывает изображения по вертикали'

        dst = Image.new('RGB', (im2.width, im1.height + im2.height))
        dst.paste(im1, (0, 0))
        dst.paste(im2, (0, im1.height))
        return dst

    def _find_coeffs(self,pa, pb):
    
        "коэффицикенты для трансформации"

        matrix = []
        for p1, p2 in zip(pa, pb):
            matrix.append([p1[0], p1[1], 1, 0, 0, 0, -p2[0]*p1[0], -p2[0]*p1[1]])
            matrix.append([0, 0, 0, p1[0], p1[1], 1, -p2[1]*p1[0], -p2[1]*p1[1]])

        A = np.matrix(matrix, dtype=np.float)
        B = np.array(pb).reshape(8)
        res = np.dot(np.linalg.inv(A.T * A) * A.T, B)
        return np.array(res).reshape(8)

    def _perspective_image(self,pil_image):

        "создание перспективы изображения"

        size = pil_image.size
        shift = random.uniform(0.03, 0.21) 
        new_width = size[0]-size[0]*shift    
        coeffs = self._find_coeffs(
                [(0, 0), (size[0], 0), (size[0], size[1]), (0, size[1])],
                [(0, 0), (size[0], 0), (new_width, size[1]), (0, size[1])])

        perspective = pil_image.transform((size[0]+int(size[0]*2*shift), size[1]+int(size[1]*0.1))
                                  , Image.PERSPECTIVE, coeffs,Image.BICUBIC)
        return perspective

    def _bending_image(self,pil_image):  

        "создание полуизгиба изображения"

        size = np.asarray(pil_image).shape
        crop_up = pil_image.crop((0,0,size[1],size[0]//2)) # cut top part of image
        crop_down = pil_image.crop((0,size[0]//2,size[1],size[0])) # cut bottom part of image
        persp_down = self._perspective_image(crop_down)
        concat_img = self._get_concat_v(crop_up, persp_down)
        return concat_img


    def _perspective_mask(self,pil_image):  

        "создание маски для непрямого изображения"

        some_array = np.asarray(pil_image)
        gray = cv2.cvtColor(some_array, cv2.COLOR_BGR2GRAY)
        img_arr = Image.fromarray(gray)
        img_arr = ImageOps.grayscale(img_arr)
        img_arr = np.asarray(img_arr)
        img_arr = np.where(img_arr>0,255,img_arr)
        img_arr = Image.fromarray(img_arr)
        return img_arr


    def _synth_generator(self, pil_image, generation_mode = '2d'): 

        "генерация синтов в двух режимах"

        angle = random.randint(-24,24) #  рандомный угол
        zoom = random.uniform(2.0, 4.0) # рандомный зум
        mirror = random.choice([0,1]) # рандомное отзеркаливание

        c = self._random_background()
        back_shape = np.asarray(c).shape


        shape = (np.asarray(pil_image).shape)
        h = shape[0]
        w = shape[1]
        test = pil_image.resize((int(w/zoom), int(h/zoom))
                            , Image.BILINEAR )    

        if mirror == 1:
            test = test.transpose(Image.FLIP_LEFT_RIGHT)

        if generation_mode == '3d':
            aug_mode = random.choice([0,1])
            if aug_mode == 0:
                test = self._perspective_image(test)
            else:
                test = self._bending_image(test)
            mask = self._perspective_mask(test) 

        else:
            mask = Image.new('L', test.size, 255)


        mask = mask.rotate(angle, expand=True)
        test = test.rotate(angle, expand=True)
        shape = np.asarray(mask).shape

        x = (back_shape[0]-shape[0])//2
        y = (back_shape[1]-shape[1])//2
        c.paste(test, (y, x), mask)

        return c   
    
  
    def start_generation(self):   
        
        if os.path.exists(self.save_dir) == False:
            os.mkdir(self.save_dir)
        
        print ('generate 2d...')
        for i in tqdm(range(self.count_2d)):
   
            path_name = random.choice(self.images)
            img = Image.open(path_name)
            
            img = self._synth_generator(img, generation_mode = '2d')
            name = str(i)+'.png'
            save_path = os.path.join(self.save_dir, name)
            img.save(save_path)            
            
        print ('generate 3d...')    
        for i in tqdm(range(self.count_3d)):
   
            path_name = random.choice(self.images)
            img = Image.open(path_name)
         
            img = self._synth_generator(img, generation_mode = '3d')
            name = str(i+self.count_2d)+'.png'
            save_path = os.path.join(self.save_dir, name)
            img.save(save_path)
            